In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import re
from bs4 import BeautifulSoup
import unicodedata

In [ ]:
!pip install contractions

In [ ]:
import contractions

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie_reviews.csv.bz2', compression='bz2')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df['sentiment'] = [1 if sentiment == 'positive' else 0 for sentiment in df['sentiment'].values]
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
df['n_sentiment'] = [0 if sentiment == 1 else 1 for sentiment in df['sentiment'].values]
df.head()

,review,sentiment,n_sentiment
0,One of the other reviewers has mentioned that ...,1,0
1,A wonderful little production. <br /><br />The...,1,0
2,I thought this was a wonderful way to spend ti...,1,0
3,Basically there's a family where a little boy ...,0,1
4,"Petter Mattei's ""Love in the Time of Money"" is...",1,0


# Creating Datasets for Training, Testing and Validation:

In [ ]:
reviews = df['review'].values
labels = df[['sentiment','n_sentiment']].values

train_reviews = reviews[:30000]
train_labels = labels[:30000]

test_reviews = reviews[30000:40000]
test_labels = labels[30000:40000]

val_reviews = reviews[45000:]
val_labels = labels[45000:]

train_reviews.shape, test_reviews.shape, val_reviews.shape,

((30000,), (10000,), (5000,))

# Functions

In [ ]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    text_ = soup.get_text()
    text_ = re.sub(r'[\r|\n|\r\n]+', '\n', text_)
    return text_

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def clean_doc(document):
    
    # strip HTML
    doc = strip_html_tags(document)
    
    # lower case
    doc = doc.lower()
    
    # remove extra newlines (often might be present in really noisy text)
    doc = doc.translate(document.maketrans("\n\t\r", "   "))
    
    # remove accented characters
    doc = remove_accented_chars(doc)
    
    # expand contractions    
    doc = expand_contractions(doc)
               
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    doc = special_char_pattern.sub(" \\1 ", doc)
    doc = remove_special_characters(doc, remove_digits=True)  
        
    # remove extra whitespace
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    
    return doc

In [ ]:
clean_corpus = np.vectorize(clean_doc)

# Pre Processing

In [ ]:
train_reviews = clean_corpus(train_reviews)
val_reviews = clean_corpus(val_reviews)
test_reviews = clean_corpus(test_reviews)

In [ ]:
train_reviews[0]

'one of the other reviewers has mentioned that after watching just oz episode you will be hooked they are right as this is exactly what happened with me the first thing that struck me about oz was its brutality and unflinching scenes of violence which set in right from the word go trust me this is not a show for the faint hearted or timid this show pulls no punches with regards to drugs sex or violence its is hardcore in the classic use of the word it is called oz as that is the nickname given to the oswald maximum security state penitentary it focuses mainly on emerald city an experimental section of the prison where all the cells have glass fronts and face inwards so privacy is not high on the agenda them city is home to many aryans muslims gangstas latinos christians italians irish and more so scuffles death stares dodgy dealings and shady agreements are never far away i would say the main appeal of the show is due to the fact that it goes where other shows would not dare forget pre

# Preparing Training

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
modelHub = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return modelHub(input)

module https://tfhub.dev/google/universal-sentence-encoder-large/5 loaded


In [ ]:
def embed_batch(input,batch_size=128):
  return_array = []
  chunks = (len(input) - 1) // batch_size + 1
  for i in range(chunks):
      batch = modelHub(input[i*batch_size:(i+1)*batch_size])
      if len(return_array) == 0:
        return_array = np.array(batch)
      else:
        return_array = np.concatenate((return_array, np.array(batch)[:,:]), axis=0)
  return return_array

In [ ]:
train_input = embed_batch(train_reviews,64)
print('Foi o primeiro')
val_input = embed_batch(val_reviews,64)
print('Foi o segundo')

Foi o primeiro
Foi o segundo


In [ ]:
from tensorflow.python.keras.layers import Embedding, Reshape, Activation, Input, Lambda, Dense, Attention, GRU, LSTM, CuDNNLSTM, CuDNNGRU, Dropout, TimeDistributed, RepeatVector
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Dense(512,input_shape=(None, train_input.shape[1]), activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='sigmoid',name='classification'))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, None, 512)         262656    
_________________________________________________________________
dense_4 (Dense)              (None, None, 256)         131328    
_________________________________________________________________
dense_5 (Dense)              (None, None, 128)         32896     
_________________________________________________________________
classification (Dense)       (None, None, 2)           258       
Total params: 427,138
Trainable params: 427,138
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(),'accuracy'])
callback_early_stopping = EarlyStopping(monitor='val_loss',patience=20,verbose=1)

In [ ]:
n_epochs = 500

In [ ]:
model.fit(
    train_input, train_labels,
    epochs=n_epochs,
    batch_size=128,
    validation_data=(val_input, val_labels),
    callbacks=[callback_early_stopping]
)

Epoch 1/500


230/235 [============================>.] - ETA: 0s - loss: 0.3662 - auc_2: 0.8788 - accuracy: 0.8334WARNING:tensorflow:Model was constructed with shape (None, None, 512) for input KerasTensor(type_spec=TensorSpec(shape=(None, None, 512), dtype=tf.float32, name='dense_3_input'), name='dense_3_input', description="created by layer 'dense_3_input'"), but it was called on an input with incompatible shape (None, 512).


235/235 [==============================] - 3s 8ms/step - loss: 0.3656 - auc_2: 0.8797 - accuracy: 0.8340 - val_loss: 0.3784 - val_auc_2: 0.9169 - val_accuracy: 0.8376
Epoch 2/500
235/235 [==============================] - 1s 6ms/step - loss: 0.3040 - auc_2: 0.9242 - accuracy: 0.8697 - val_loss: 0.2990 - val_auc_2: 0.9303 - val_accuracy: 0.8686
Epoch 3/500
235/235 [==============================] - 1s 6ms/step - loss: 0.2735 - auc_2: 0.9353 - accuracy: 0.8863 - val_loss: 0.2977 - val_auc_2: 0.9388 - val_accuracy: 0.8744
Epoch 4/500
235/235 [==============================] - 1s 6ms/step - loss: 0.2327 - auc_2: 0.9427 - accuracy: 0.9056 - val_loss: 0.3195 - val_auc_2: 0.9459 - val_accuracy: 0.8678
Epoch 5/500
235/235 [==============================] - 1s 6ms/step - loss: 0.1949 - auc_2: 0.9492 - accuracy: 0.9205 - val_loss: 0.3563 - val_auc_2: 0.9519 - val_accuracy: 0.8558
Epoch 6/500
235/235 [==============================] - 2s 6ms/step - loss: 0.1595 - auc_2: 0.9548 - accuracy: 0.9362 

In [ ]:
print('Importando test_input...')
test_input = embed_batch(test_reviews,32)

Importando test_input...


In [ ]:
model.evaluate(test_input,test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 1.1811 - auc_2: 0.9818 - accuracy: 0.8454


[1.1811271905899048, 0.9818480610847473, 0.8453999757766724]